In [9]:
import numpy as np
import pandas as pd

import xgboost
from sklearn.preprocessing import LabelEncoder

# Import dataset

In [10]:
data = pd.read_csv("../DataFormating/compressed_final.csv")

# Set `X` and `y`

In [11]:
data.columns[:10]

Index(['Stage', 'Home Team Name', 'Home Team Goals', 'Away Team Goals',
       'Away Team Name', 'Attendance', 'Half-time Home Goals',
       'Half-time Away Goals', 'Home Team Initials', 'Away Team Initials'],
      dtype='object')

In [12]:
X = data.drop(["Home Team Goals", "Away Team Goals",
               "Half-time Home Goals", "Half-time Away Goals", 
               "Home Team Initials", "Away Team Initials"], axis=1)

y = []
for i in range(len(data)):
    home_team_goals = data.iloc[i]["Home Team Goals"]
    away_team_goals = data.iloc[i]["Away Team Goals"]
    
    if home_team_goals > away_team_goals:
        y.append(1)
    elif home_team_goals < away_team_goals:
        y.append(2)
    else:
        y.append(0)

In [13]:
# Test
assert len(X) == len(y)

### Encode textual features from the `X` dataset

In [14]:
stage_encoder = LabelEncoder().fit(X["Stage"])
home_team_name_encoder = LabelEncoder().fit(X["Home Team Name"])
away_team_name_encoder = LabelEncoder().fit(X["Away Team Name"])

In [15]:
X["Stage"] = stage_encoder.transform(X["Stage"])
X["Home Team Name"] = home_team_name_encoder.transform(X["Home Team Name"])
X["Away Team Name"] = away_team_name_encoder.transform(X["Away Team Name"])

### Feature Selection

In [16]:
feature_names = [
    "Stage", "Home Team Name", "Away Team Name",
    "Attendance", "Overall",
    "Mean Home Team Goals", "Mean Away Team Goals"
]

COLUMNS = []

for column_name in X.columns:
    for feature_name in feature_names:
        if feature_name in column_name:
            COLUMNS.append(column_name)
            break

X = X[COLUMNS]